In [14]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

from matplotlib import font_manager,rc

rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus']=False     # 마이너스 부호 깨짐 방지
#현재 폴더 경로; 작업 폴더 기준
pd.options.display.float_format = '{:.1f}'.format
print(os.getcwd())

C:\cakd7\git\2차 프로젝트\second_projects\second_projects\생성데이터


# 가중치 적용한 데이터

## F_카테고리 AMOUNT TOP2 구하기

In [12]:
# 1분기
year2014_q1 = pd.read_csv('생성데이터/weight_plus_2014_q1.csv', index_col = 0)
year2014_q1['purchase_sum'] = year2014_q1.apply(lambda x:x.sum(), axis = 1)

# 2분기
year2014_q2 = pd.read_csv('생성데이터/weight_plus_2014_q2.csv', index_col = 0)
year2014_q2['purchase_sum'] = year2014_q2.apply(lambda x:x.sum(), axis = 1)

# 3분기
year2014_q3 = pd.read_csv('생성데이터/weight_plus_2014_q3.csv', index_col = 0)
year2014_q3['purchase_sum'] = year2014_q3.apply(lambda x:x.sum(), axis = 1)

#4분기
year2014_q4 = pd.read_csv('생성데이터/weight_plus_2014_q4.csv', index_col = 0)
year2014_q4['purchase_sum'] = year2014_q4.apply(lambda x:x.sum(), axis = 1)

#5분기
year2015_q1 = pd.read_csv('생성데이터/weight_plus_2015_q1.csv', index_col = 0)
year2015_q1['purchase_sum'] = year2015_q1.apply(lambda x:x.sum(), axis = 1)

# 6분기
year2015_q2 = pd.read_csv('생성데이터/weight_plus_2015_q2.csv', index_col = 0)
year2015_q2['purchase_sum'] = year2015_q2.apply(lambda x:x.sum(), axis = 1)

#7분기
year2015_q3 = pd.read_csv('생성데이터/weight_plus_2015_q3.csv', index_col = 0)
year2015_q3['purchase_sum'] = year2015_q3.apply(lambda x:x.sum(), axis = 1)

#8분기
year2015_q4 = pd.read_csv('생성데이터/weight_plus_2015_q4.csv', index_col = 0)
year2015_q4['purchase_sum'] = year2015_q4.apply(lambda x:x.sum(), axis = 1)

FileNotFoundError: [Errno 2] No such file or directory: '생성데이터/weight_plus_2014_q1.csv'

In [ ]:
df_li = [year2014_q2, year2014_q3, year2014_q4, year2015_q1, year2015_q2, year2015_q3, year2015_q4]

df_ = year2014_q1
for i in df_li:
    display(df_)
    df_ += i
    
df_.head()

In [ ]:
# 빈 데이터 프레임
total_df = pd.DataFrame(index = year2014_q1.index)


df = df_.drop('purchase_sum', axis=1)

# 고객 번호별로 중요 상품군 추가됨
for cust in df.index:
    # 고객 번호 별로 담겨질 딕셔너리
    dic_1 = {} # top1
    dic_2 = {} # top2

    rank_top2 = df.loc[cust].sort_values(ascending = False)[:2] 

    # 딕셔너리로 차례대로 담기 (해당 카테고리 : 구매금액)
    dic_1[f'{rank_top2.index[0]}'] = rank_top2[0]
    dic_2[f'{rank_top2.index[1]}'] = rank_top2[1]

    # total_df에 추가
    column_top1 = '전체 분기 중요 카테고리 top_1'
    column_top2 = '전체 분기 중요 카테고리 top_2'

    total_df.loc[cust, column_top1] = [dic_1]
    total_df.loc[cust, column_top2] = [dic_2]
total_df

# R 평균 구매주기 

In [ ]:
from datetime import datetime

def out_q_df(df):
    # 고객
    cust_li = df['CUSTNO'].unique().tolist()
    cust_li.sort()
    # 카테고리
    cate_li = df['TOTAL_CAT'].unique().tolist()
    cate_li.sort()
    
    # 최종 df 
    df_t = pd.DataFrame(index = cust_li, columns = cate_li)
    
    # 고객별, 카테고리별 df 반출
    for cate in cate_li:
        cate_df = df[df['TOTAL_CAT']==cate]
        
        # 고객리스트
        cust_li_ = cate_df['CUSTNO'].unique().tolist()
        cust_li_.sort()
        
        for cust in cust_li_:
            cust_df = cate_df[cate_df['CUSTNO']==cust]

            cat_date = cust_df['PURDATE'].unique().tolist()
            cat_date.sort(reverse=True)

            if len(cat_date) == 0:
                pass
            elif len(cat_date) == 1:
                mean_date = -1 #'just_one'
                df_t.loc[cust, cate] = mean_date
            else:
                date_li = []
                for day_ind in range(len(cat_date)-1):
                    day_after = cat_date[day_ind]
                    day_before = cat_date[day_ind + 1]

                    # 다음구매일 - 전구매일
                    date = datetime.strptime(str(day_after),'%Y%m%d') - datetime.strptime(str(day_before),'%Y%m%d')

                    date_li.append(date.days)
                # 평균 구매일    
                mean_date = sum(date_li) / len(date_li)

                df_t.loc[cust, cate] = mean_date
    return df_t

In [ ]:
# 전체
def out_q_total_df(q_df, add_df):
    
    # 고객리스트
    for cust in q_df['CUSTNO'].unique().tolist():
        cust_df = q_df[q_df['CUSTNO']==cust]

        cat_date = cust_df['PURDATE'].unique().tolist()
        cat_date.sort(reverse=True)

        if len(cat_date) == 0:
            pass
        elif len(cat_date) == 1:
            mean_date = -1 #'just_one'
            add_df.loc[cust, '총 평균 구매주기'] = mean_date
        else:
            date_li = []
            for day_ind in range(len(cat_date)-1):
                day_after = cat_date[day_ind]
                day_before = cat_date[day_ind + 1]

                # 다음구매일 - 전구매일
                date = datetime.strptime(str(day_after),'%Y%m%d') - datetime.strptime(str(day_before),'%Y%m%d')

                date_li.append(date.days)
            # 평균 구매일    
            mean_date = sum(date_li) / len(date_li)

            add_df.loc[cust, '총 평균 구매주기'] = mean_date
    return add_df